# EDA

In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

df = pd.read_csv('/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train.csv')

In [10]:
df.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [11]:
unique_labels = df['label_group'].unique()
len(unique_labels)

11014

In [12]:
df.groupby('label_group').count().sort_values(by='posting_id', ascending=False)

,posting_id,image,image_phash,title
label_group,,,,
1163569239,51,51,51,51
159351600,51,51,51,51
994676122,51,51,51,51
3113678103,51,51,51,51
3627744656,51,51,51,51
...,...,...,...,...
2357508171,2,2,2,2
2357372960,2,2,2,2
2357221297,2,2,2,2


In [13]:
# check title length range by word
df['title'].str.split().apply(lambda x: len(x)).describe()

count    34250.000000
mean         9.414861
std          4.446719
min          1.000000
25%          6.000000
50%          9.000000
75%         12.000000
max         61.000000
Name: title, dtype: float64

In [17]:
df['title'].nunique()

33117

In [15]:
# check the id count
df['posting_id'].nunique()

34250

In [16]:
len(df['posting_id'])

34250

In [9]:
df["image_phash"].nunique() / len(df["image_phash"])

0.838978102189781

In [1]:
# plot the distribution of image_phash
# df["image_phash"].value_counts().plot(kind='hist', bins=20)


In [20]:
PATH_TO_IMG = '/workspaces/Shopee-Price-Match-Guarantee/00_source_data/shopee-product-matching/train_images/'

In [ ]:
def show_same_img():
    # choose randomly two instances per each class
    labels_to_show = np.random.choice(df.label_group.unique(), 
                                      replace=False, size=27)
    img_to_show = []
    for label in labels_to_show:
        rows = df[df.label_group==label].copy()
        pair = np.random.choice([i for i in range(len(rows))], 
                                    replace=False, size=2)
        img_pair = rows.iloc[pair][['image', 'title']].values
        
        img_to_show += list(img_pair)
    
    fig, axes = plt.subplots(figsize = (18, 12), nrows=4,ncols=6)
    for imp, ax in zip(img_to_show, axes.ravel()):
        img = cv2.imread(PATH_TO_IMG + imp[0])
        title = '\n'.join(wrap(imp[1], 20))
        ax.set_title(title)
        ax.imshow(img)
        ax.axis('off')

    fig.tight_layout()